<a href="https://colab.research.google.com/github/agarwalpratik/aiml/blob/main/BoostingImplementationCancerSocial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Create a model that can classify the detected cancer as Benign(B) and Malignant(M)

In [1]:
#XGBoost is not a part of Sklearn package

import xgboost
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('cancer.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 569 non-null    int64  
 1   diagnosis          569 non-null    object 
 2   radius_mean        569 non-null    float64
 3   texture_mean       569 non-null    float64
 4   perimeter_mean     569 non-null    float64
 5   area_mean          569 non-null    float64
 6   smoothness_mean    569 non-null    float64
 7   compactness_mean   569 non-null    float64
 8   concavity_mean     569 non-null    float64
 9   points_mean        569 non-null    float64
 10  symmetry_mean      569 non-null    float64
 11  dimension_mean     569 non-null    float64
 12  radius_se          569 non-null    float64
 13  texture_se         569 non-null    float64
 14  perimeter_se       569 non-null    float64
 15  area_se            569 non-null    float64
 16  smoothness_se      569 non

In [4]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,87139402,B,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,...,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,8910251,B,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,...,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,905520,B,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,...,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,868871,B,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,...,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,9012568,B,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,...,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766


In [5]:
data.diagnosis.value_counts()

,count
diagnosis,
B,357
M,212


In [6]:
features = data.iloc[:,2:32].values
label = data.iloc[:,1].values


In [7]:
label

array(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'M', 'B', 'B',
       'B', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'M',
       'B', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'B', 'M',
       'M', 'M', 'M', 'M', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'B',
       'B', 'B', 'M', 'B', 'B', 'B', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'M', 'B',
       'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'B', 'B',
       'B', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'B', 'M',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'M',
       'M', 'M', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'M', 'M', 'M',
       'M', 'B', 'B', 'M', 'B', 'M', 'B', 'B', 'M', 'B', 'M', 'M', 'M',
       'M', 'M', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'B',
       'B', 'M', 'M', 'M', 'B', 'M', 'B', 'M', 'B', 'M', 'B', 'B

In [8]:
#XGBoost expects the label column to be represented in the form of numeric data (In classification , its DISCRETE NUMERICAL DATA)
#Converting categorical label to discrete numerical label, you may use LabelEncoder in sklearn package

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(label)


In [9]:
label

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,

In [10]:
le.classes_

array(['B', 'M'], dtype=object)

In [11]:
le.transform(np.array(['M']))

array([1])

In [12]:
le.inverse_transform(np.array([1]))

array(['M'], dtype=object)

In [ ]:
#Boosting
# XGboost can be used for Supervised learning only
# XGBoost internally uses DecisionTree algo as base estimator
# Classification :   XGBClassifier  | XGBRFClassifier
# Regression : XGBRegressor | XGBRFRegressor

In [13]:
#pip install xgboost

In [14]:
CL = 0.94
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

for i in range(1,101):
  X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.2,random_state=i)
  model = XGBClassifier()
  model.fit(X_train,y_train)
  testScore = model.score(X_test,y_test)
  trainScore = model.score(X_train,y_train)

  if testScore > trainScore and testScore > CL:
    print(f"Test Score : {testScore} TrainScore : {trainScore} RS: {i}")

In [21]:
CL = 0.94
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

for i in range(1,101):
  X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.2,random_state=i)
  for md in range(3,20):
    model = XGBClassifier(max_depth=md,learning_rate=0.002)
    model.fit(X_train,y_train)
    testScore = model.score(X_test,y_test)
    trainScore = model.score(X_train,y_train)

    if testScore > trainScore and testScore > CL:
      print(f"Test Score : {testScore} TrainScore : {trainScore} RS: {i} Max_Depth: {md}")

In [16]:
CL = 0.94
from xgboost import XGBRFClassifier
from sklearn.model_selection import train_test_split

for i in range(1,101):
  X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.2,random_state=i)
  model = XGBRFClassifier()
  model.fit(X_train,y_train)
  testScore = model.score(X_test,y_test)
  trainScore = model.score(X_train,y_train)

  if testScore > trainScore and testScore > CL:
    print(f"Test Score : {testScore} TrainScore : {trainScore} RS: {i}")

Test Score : 0.9912280701754386 TrainScore : 0.9868131868131869 RS: 44
Test Score : 1.0 TrainScore : 0.989010989010989 RS: 72
Test Score : 0.9912280701754386 TrainScore : 0.9846153846153847 RS: 99


In [22]:
#XGBoost is not a part of Sklearn package

import xgboost
import numpy as np
import pandas as pd

In [23]:
data = pd.read_csv('Social_Network_Ads.csv')

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Age              400 non-null    int64
 1   EstimatedSalary  400 non-null    int64
 2   Purchased        400 non-null    int64
dtypes: int64(3)
memory usage: 9.5 KB


In [25]:
data.head()

,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0


In [26]:
data.Purchased.value_counts()

,count
Purchased,
0,257
1,143


In [30]:
features = data.iloc[:,0:2].values
label = data.iloc[:,2].values


In [31]:
features

array([[    19,  19000],
       [    35,  20000],
       [    26,  43000],
       [    27,  57000],
       [    19,  76000],
       [    27,  58000],
       [    27,  84000],
       [    32, 150000],
       [    25,  33000],
       [    35,  65000],
       [    26,  80000],
       [    26,  52000],
       [    20,  86000],
       [    32,  18000],
       [    18,  82000],
       [    29,  80000],
       [    47,  25000],
       [    45,  26000],
       [    46,  28000],
       [    48,  29000],
       [    45,  22000],
       [    47,  49000],
       [    48,  41000],
       [    45,  22000],
       [    46,  23000],
       [    47,  20000],
       [    49,  28000],
       [    47,  30000],
       [    29,  43000],
       [    31,  18000],
       [    31,  74000],
       [    27, 137000],
       [    21,  16000],
       [    28,  44000],
       [    27,  90000],
       [    35,  27000],
       [    33,  28000],
       [    30,  49000],
       [    26,  72000],
       [    27,  31000],


In [34]:
CL = 0.90
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

for i in range(1,101):
  X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.2,random_state=i)
  model = XGBRegressor()
  model.fit(X_train,y_train)
  testScore = model.score(X_test,y_test)
  trainScore = model.score(X_train,y_train)

  if testScore > trainScore and testScore > CL:
    print(f"Test Score : {testScore} TrainScore : {trainScore} RS: {i}")

In [35]:
CL = 0.90
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

for i in range(1,101):
  X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.2,random_state=i)
  for md in range(3,20):
    model = XGBRegressor(max_depth=md,learning_rate=0.002)
    model.fit(X_train,y_train)
    testScore = model.score(X_test,y_test)
    trainScore = model.score(X_train,y_train)

    if testScore > trainScore and testScore > CL:
      print(f"Test Score : {testScore} TrainScore : {trainScore} RS: {i} Max_Depth: {md}")

In [37]:
CL = 0.94
from xgboost import XGBRFRegressor
from sklearn.model_selection import train_test_split

for i in range(1,101):
  X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.2,random_state=i)
  model = XGBRFRegressor()
  model.fit(X_train,y_train)
  testScore = model.score(X_test,y_test)
  trainScore = model.score(X_train,y_train)

  if testScore > trainScore and testScore > CL:
    print(f"Test Score : {testScore} TrainScore : {trainScore} RS: {i}")